## Reading Excel files

This notebook demonstrates how to read and manipulate data from
Excel using [Pandas](http://pandas.pydata.org/):

* Input / Output
* summaries
* plotting


###### sausage making
There are ugly details related to formatting that come with the territory of Excel files. It is possible to handle them with Pandas, but discouraging. I have cleaned up the Excel file by hand and put it here:
[[ TANYA PUT THE LINK HERE]]

In [ ]:
# The library for handling tabular data is called 'pandas'
# Everyone shortens this to 'pd' for convenience.
import pandas as pd

## Get IRS data on businesses

The IRS website has some [aggregated statistics on business returns][irs] in Excel files. We will use the [Selected Income and Tax Items for Selected Years][intaba14].

We will use the file linked here:
https://www.irs.gov/pub/irs-soi/14intaba.xls.
It looks like this:
<img src="img/screenshot-14intaba.png" width="100%"/>

[irs]: https://www.irs.gov/uac/soi-tax-stats-integrated-business-data
[intaba14]: https://www.irs.gov/uac/soi-tax-stats-individual-time-series-statistical-tables#_grp4

###  Sausage!

We will use the `read_excel` function inside of the Pandas library (accessed using `pd.read_excel`) to get the data. Then we'll:

- skip the first 2 rows
  * groups of columns (denoting current or 2007 dollars) are on row 2,
  * and the years are on row 3
- remove the footnote rows
- Extract the two levels of information in Column A (denoted by indentation)

In [43]:
# IRS business data
# We can pull it directly from the internet
file_location = 'https://www.irs.gov/pub/irs-soi/14intaba.xls'

raw = pd.read_excel(file_location, skiprows=2)
raw.head()

,Item [1],Current dollars,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,"Percent change, 2013 to 2014.1",Unnamed: 53
0,NaN,1990.0,1991,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,...,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.0,2,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52,NaN
4,All returns,113717138.0,114730123,113604503.0,114601819.0,115943131.0,118218327.0,120351208.0,122421991.0,124770662.0,...,142978806.0,142450569.0,140494127.0,142892051.0,145370240.0,144928472.0,147351299.0,148606578.0,0.851895,NaN


In [48]:
#data['new_index'] = range(1980,2013)
new_columns = [ zip(raw.columns, raw.loc[0]) ]
years = raw.loc[0]

Item [1]                            NaN
Current dollars                    1990
Unnamed: 2                         1991
Unnamed: 3                         1992
Unnamed: 4                         1993
Unnamed: 5                         1994
Unnamed: 6                         1995
Unnamed: 7                         1996
Unnamed: 8                         1997
Unnamed: 9                         1998
Unnamed: 10                        1999
Unnamed: 11                        2000
Unnamed: 12                        2001
Unnamed: 13                        2002
Unnamed: 14                        2003
Unnamed: 15                        2004
Unnamed: 16                        2005
Unnamed: 17                        2006
Unnamed: 18                        2007
Unnamed: 19                        2008
Unnamed: 20                        2009
Unnamed: 21                        2010
Unnamed: 22                        2011
Unnamed: 23                        2012
Unnamed: 24                        2013


In [18]:
# Remove the first 3 rows of data
data = 
data.head()

,0,1,2,3,4,5,7,8,9,10,...,58,59,60,61,63,64,65,66,67,68
All Businesses,Number of returns [6],Total receipts,Business receipts,Net income (less deficit),Net income,Deficit,Number of returns [6],Total receipts,Business receipts,Net income (less deficit) [1],...,Business receipts,Net income (less deficit),Net income,Deficit,Number of returns [6],Total receipts,Business receipts,Net income (less deficit),Net income,Deficit
Unnamed: 1,13021904,7064487840,6413930882,316874165,424569277,107695112,2710538,6361284012,5731616337,253678291,...,n.a.,n.a.,n.a.,n.a.,8931712,411205713,411205713,54947219,68010051,13062832
Unnamed: 2,13857712,7725544701,6901768455,263985693,420560759,156575064,2812420,7026351839,6244678064,213648962,...,n.a.,n.a.,n.a.,n.a.,9584790,427063055,427063055,53071628,68552791,15481162
Unnamed: 3,14545660,7754452966,6842267893,197592719,396557182,198964461,2925933,7024097766,6156994009,154334143,...,n.a.,n.a.,n.a.,n.a.,10105515,433664897,433664897,50573163,68647384,18074220
Unnamed: 4,15244531,7891981399,7043019718,246063040,435858670,189795629,2999071,7135494059,6334602711,188313928,...,n.a.,n.a.,n.a.,n.a.,10703921,465168637,465168637,60359153,78618410,18259256


We can remove the first 3 rows of data.